# Medical Notes Classification

Medical notes is an useful information source for patient data extraction. Notes classification is also an important task in Medical NLP domain. There are many techniques to solve this problem ranging from traditional method (Logistic Regression, SVM,...) to the state-of-the-art models (Transformer).

The below code block is the baseline model for a text classification problem in medical domain.

* Input: the corpus of medical transcriptions.
* Output: the type of each notes.

In this problem, we try to classify five labels:
* Surgery
* Consult - History and Phy.
* Cardiovascular / Pulmonary
* Orthopedic
* Others

The train-test split was also defined, please don't change our split.

Metric to evaluate: `f1_macro`

# Baseline Model Result


0.3729330560342061

                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary       0.35      0.39      0.37       148
    Consult - History and Phy.       0.32      0.06      0.10       207
                    Orthopedic       0.39      0.14      0.21       142
                         Other       0.66      0.74      0.70      1055
                       Surgery       0.43      0.57      0.49       435

                      accuracy                           0.56      1987
                     macro avg       0.43      0.38      0.37      1987
                  weighted avg       0.54      0.56      0.53      1987

# Library & Data Loading

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics


os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


### PLEASE DON'T CHANGE ANYTHING IN THIS SECTION ###
DATA = "https://github.com/socd06/private_nlp/raw/master/data/mtsamples.csv"

filtered_labels = [
    "Surgery",
    "Consult - History and Phy.",
    "Cardiovascular / Pulmonary",
    "Orthopedic",
]
data = pd.read_csv(DATA, usecols=['medical_specialty', 'transcription']).dropna()
data.columns = ['labels', 'text']
data['labels'] = [i.strip() if (i.strip() in filtered_labels) else 'Other' for i in data.labels.to_list()]
train, test = train_test_split(data, test_size=0.4, stratify=data.labels, random_state=0)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
### END ###

# Preprocessing

# My Model


In [2]:
#encode labels 
le = LabelEncoder()
train['labels'] = le.fit_transform(train.labels)
test['labels'] = le.transform(test.labels)

### Encode Text


In [3]:
vector_size = 500
segment_size = 20
data_enrichment = 3

In [4]:
#create model
import TopicAllocate as ta
model = ta.Topic_Allocate()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haimi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haimi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#encode texts into matrix
X_train = np.asarray(model.doc2vec(train['text'], vector_size = vector_size, segment_size = segment_size, data_enrichment = data_enrichment, fit = True))
X_test = np.asarray(model.doc2vec(test['text'], vector_size = vector_size, segment_size = segment_size, data_enrichment = data_enrichment))


C:\Users\haimi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [6]:
# onehot labels
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

y_train = train['labels']

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
Y_train = onehot_encoder.fit_transform(np.array(y_train).reshape(-1, 1))
Y_test = onehot_encoder.transform(np.array(test['labels']).reshape(-1,1))

# invert first example
inverted = le.inverse_transform([np.argmax(Y_train[0, :])])
print(inverted)

['Consult - History and Phy.']


### Train with LSTM

In [7]:
#%% import library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [8]:
#remove some noise
text_len = np.array([len(text.split()) for text in train['text']]) 
text_test_len = np.array([len(text.split()) for text in test['text']]) 

In [9]:
other_label = np.where(np.argmax(Y_train, axis = 1) == 3)[0]
res_label = np.where(np.argmax(Y_train, axis = 1) != 3)[0]
train_keep = np.where(text_len > 30)[0]
#cut less of other topic
other_label = np.random.choice(other_label,int (other_label.shape[0] / 3 * 2))
train_keep =np.array( list(set(np.hstack((other_label,res_label))).intersection(train_keep)))
print(train_keep.shape[0])

2116


In [10]:
max_size = np.amax(np.array([x.shape[0] for x in X_train]))
def fill_zeros(x, Vector_size):
    try:
        missing = max_size - x.shape[0]
        fill_in = np.zeros((missing, Vector_size))
        return np.vstack((fill_in, x))
    except:
        return np.zeros((max_size, Vector_size))
func = lambda x: fill_zeros(x, 500)
X_train_lstm_s2v = np.array([func(x) for x in X_train])
X_test_lstm_s2v = np.array([func(x) for x in X_test])


In [11]:
#%% Xay dung model LSTM
lstm_model = Sequential()
# return_sequences: tra lai ket qua cuoi cho lop tiep theo
lstm_model.add(LSTM(units=200, return_sequences=True, input_shape=(X_train_lstm_s2v.shape[1], X_train_lstm_s2v.shape[2])))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=5, activation="softmax"))
lstm_model.compile(optimizer="adam", loss="categorical_crossentropy",  metrics=["accuracy"])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 121, 200)          560800    
_________________________________________________________________
dropout (Dropout)            (None, 121, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                50200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 255       
Total params: 611,255
Trainable params: 611,255
Non-trainable params: 0
_________________________________________________________________


In [12]:
lstm_model.fit(X_train_lstm_s2v[train_keep], Y_train[train_keep], epochs=20, batch_size=256, validation_split= 0.1)

Epoch 1/20
8/8 [==============================] - 17s 623ms/step - loss: 1.5041 - accuracy: 0.4065 - val_loss: 1.4019 - val_accuracy: 0.4858
Epoch 2/20
8/8 [==============================] - 2s 211ms/step - loss: 1.3080 - accuracy: 0.4937 - val_loss: 1.2662 - val_accuracy: 0.4764
Epoch 3/20
8/8 [==============================] - 2s 214ms/step - loss: 1.2371 - accuracy: 0.5116 - val_loss: 1.2397 - val_accuracy: 0.5047
Epoch 4/20
8/8 [==============================] - 2s 223ms/step - loss: 1.1933 - accuracy: 0.5236 - val_loss: 1.2114 - val_accuracy: 0.4858
Epoch 5/20
8/8 [==============================] - 2s 208ms/step - loss: 1.1708 - accuracy: 0.5163 - val_loss: 1.1942 - val_accuracy: 0.4717
Epoch 6/20
8/8 [==============================] - 2s 228ms/step - loss: 1.1391 - accuracy: 0.5305 - val_loss: 1.1630 - val_accuracy: 0.4906
Epoch 7/20
8/8 [==============================] - 2s 221ms/step - loss: 1.1318 - accuracy: 0.5263 - val_loss: 1.1556 - val_accuracy: 0.5047
Epoch 8/20
8/8 [===

In [13]:
from sklearn.metrics import confusion_matrix
check = lstm_model.predict(X_test_lstm_s2v)
check = np.argmax(check, axis = 1)
ytrain = np.argmax(Y_test, axis = 1)
print(confusion_matrix(check, ytrain))
print(metrics.f1_score(ytrain, check, average='macro'))
print(metrics.classification_report(ytrain, check, target_names=list(le.classes_)))

[[ 51   6   0  42  17]
 [ 10  94  10 126   1]
 [  0   5  72  67  54]
 [ 25 102  21 516   8]
 [ 62   0  39 304 355]]
0.48424924034878175
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.44      0.34      0.39       148
Consult - History and Phy.       0.39      0.45      0.42       207
                Orthopedic       0.36      0.51      0.42       142
                     Other       0.77      0.49      0.60      1055
                   Surgery       0.47      0.82      0.59       435

                  accuracy                           0.55      1987
                 macro avg       0.49      0.52      0.48      1987
              weighted avg       0.61      0.55      0.55      1987

